# MPP Sequencing data

Input MPP metadata for the sequencing data into Mediaflux. 
Each library is archived (zipped with no compression) and uploaded to mediaflux. 
Each archive has metadata attached indicating the samples sequenced and barcodes for each sample.

In [1]:
import sys
import logging
from Crypto.Cipher import AES
import pandas as pd
import random
import re
import os
import datetime
import time

In [2]:
sys.path.insert(0, 'python-mfclient/src')
import mfclient

In [3]:
logging.basicConfig(
    filename="mf_2018-02-13.log",
    level=logging.DEBUG,
    filemode="a",
    format='%(asctime)s %(levelname)s - %(message)s',
    datefmt='%m-%d-%Y %H:%M:%S')

-----

## Parse text and load files

In [4]:
raw_text = """
2016-10-30_Burramys_MtBuller_RADSeq_MiSeq/miseq_barcode.txt
2017-03-24_Burramys_MtBuller_RADSeq_Lib1_trial/lib1_trial_barcode.txt
2017-08-29_Burramys_MtBuller_RADSeq_Lib1/lib1_barcode.txt
2017-08-29_Burramys_MtBuller_RADSeq_Lib2/lib2_barcode.txt
2017-08-29_Burramys_MtBuller_RADSeq_Lib3/lib3_barcode.txt
2017-08-29_Burramys_MtBuller_RADSeq_Lib4/lib4_barcode.txt
2017-08-29_Burramys_MtBuller_RADSeq_Lib5/lib5_barcode.txt
2017-08-29_Burramys_MtBuller_RADSeq_Lib6/lib6_barcode.txt
2017-08-29_Burramys_MtBuller_RADSeq_Lib7/lib7_barcode.txt
2017-10-31_Burramys_MtBuller_RADSeq_Lib10/lib10_barcode.txt
2017-10-31_Burramys_MtBuller_RADSeq_Lib8/lib8_barcode.txt
2017-10-31_Burramys_MtBuller_RADSeq_Lib9/lib9_barcode.txt
2018-01-31_Burramys_MtBuller_RADSeq_Lib11/lib11_barcode.txt
2018-01-31_Burramys_MtBuller_RADSeq_Lib12/lib12_barcode.txt
"""

In [5]:
text_list = raw_text.strip().split("\n")

In [6]:
library_names = [os.path.dirname(x) for x in text_list]
library_names

['2016-10-30_Burramys_MtBuller_RADSeq_MiSeq',
 '2017-03-24_Burramys_MtBuller_RADSeq_Lib1_trial',
 '2017-08-29_Burramys_MtBuller_RADSeq_Lib1',
 '2017-08-29_Burramys_MtBuller_RADSeq_Lib2',
 '2017-08-29_Burramys_MtBuller_RADSeq_Lib3',
 '2017-08-29_Burramys_MtBuller_RADSeq_Lib4',
 '2017-08-29_Burramys_MtBuller_RADSeq_Lib5',
 '2017-08-29_Burramys_MtBuller_RADSeq_Lib6',
 '2017-08-29_Burramys_MtBuller_RADSeq_Lib7',
 '2017-10-31_Burramys_MtBuller_RADSeq_Lib10',
 '2017-10-31_Burramys_MtBuller_RADSeq_Lib8',
 '2017-10-31_Burramys_MtBuller_RADSeq_Lib9',
 '2018-01-31_Burramys_MtBuller_RADSeq_Lib11',
 '2018-01-31_Burramys_MtBuller_RADSeq_Lib12']

In [7]:
library_archives = [x + ".zip" for x in library_names]
library_archives[:3]

['2016-10-30_Burramys_MtBuller_RADSeq_MiSeq.zip',
 '2017-03-24_Burramys_MtBuller_RADSeq_Lib1_trial.zip',
 '2017-08-29_Burramys_MtBuller_RADSeq_Lib1.zip']

In [8]:
barcode_filenames = [os.path.basename(x) for x in text_list]
barcode_filenames

['miseq_barcode.txt',
 'lib1_trial_barcode.txt',
 'lib1_barcode.txt',
 'lib2_barcode.txt',
 'lib3_barcode.txt',
 'lib4_barcode.txt',
 'lib5_barcode.txt',
 'lib6_barcode.txt',
 'lib7_barcode.txt',
 'lib10_barcode.txt',
 'lib8_barcode.txt',
 'lib9_barcode.txt',
 'lib11_barcode.txt',
 'lib12_barcode.txt']

```
# Download barcode files
rsync -av jess@115.146.85.115:/home/elf/elf/Burramys/data/*/*barcode.txt .
```

In [9]:
# Check barcode files
for x in barcode_filenames:
    assert(x in os.listdir("../data/barcodes/"))

In [10]:
library_info = {}
for i, archive in enumerate(library_archives):
    library_info[archive] = {}
    library_info[archive]["barcode_filename"] = barcode_filenames[i]
    library_info[archive]["library_name"] = library_names[i]
    library_info[archive]["barcode_info"] = pd.read_table(
        "../data/barcodes/{}".format(library_info[archive]["barcode_filename"]), 
        header=None)

In [11]:
library_info["2017-03-24_Burramys_MtBuller_RADSeq_Lib1_trial.zip"]["barcode_info"].head()

,0,1,2
0,ACGTCA,CATGAC,Buller_969
1,GTACTG,CATGAC,Buller_972
2,CTAGTC,CATGAC,Buller_978
3,AGCTGA,CATGAC,Buller_8115
4,TCGACT,CATGAC,Buller_8120


```
# Additionally, two archives don't have barcode info, and the samples are already 
# separated into separate fastq files
#   2014_02_XX_Burramys_MtBuller_RADSeq_NlaIIIxMluCI.zip
#   2017-12-26_Burramys_MtBuller_WGS_LibHair.zip

cd /home/elf/elf/Burramys/data/2014_02_XX_Burramys_MtBuller_RADSeq_NlaIIIxMluCI
ls -1 * | tr '\n' ' '

cd /home/elf/elf/Burramys/data/2017-12-26_Burramys_MtBuller_WGS_LibHair
ls -1 * | tr '\n' ' '
```

In [17]:
NlaIIIxMluCI_files = "10_1.fq.gz 10_2.fq.gz 204_1.fq.gz 204_2.fq.gz 312_1.fq.gz 312_2.fq.gz 313_1.fq.gz 313_2.fq.gz 377_1.fq.gz 377_2.fq.gz 379_1.fq.gz 379_2.fq.gz 380_1.fq.gz 380_2.fq.gz 381_1.fq.gz 381_2.fq.gz 383_1.fq.gz 383_2.fq.gz 384_1.fq.gz 384_2.fq.gz 385_1.fq.gz 385_2.fq.gz 386_1.fq.gz 386_2.fq.gz 388_1.fq.gz 388_2.fq.gz 389_1.fq.gz 389_2.fq.gz 390_1.fq.gz 390_2.fq.gz 391_1.fq.gz 391_2.fq.gz 392_1.fq.gz 392_2.fq.gz 393_1.fq.gz 393_2.fq.gz 395_1.fq.gz 395_2.fq.gz 6654_1.fq.gz 6654_2.fq.gz 6656_1.fq.gz 6656_2.fq.gz 7323_1.fq.gz 7323_2.fq.gz 7343_1.fq.gz 7343_2.fq.gz 7473_1.fq.gz 7473_2.fq.gz 8115_1.fq.gz 8115_2.fq.gz 8120_1.fq.gz 8120_2.fq.gz 8124_1.fq.gz 8124_2.fq.gz 8164_1.fq.gz 8164_2.fq.gz 8175_1.fq.gz 8175_2.fq.gz 8185_1.fq.gz 8185_2.fq.gz 8391_1.fq.gz 8391_2.fq.gz 8426_1.fq.gz 8426_2.fq.gz 8462_1.fq.gz 8462_2.fq.gz 8470_1.fq.gz 8470_2.fq.gz 8471_1.fq.gz 8471_2.fq.gz 8473_1.fq.gz 8473_2.fq.gz 8526_1.fq.gz 8526_2.fq.gz 8647_1.fq.gz 8647_2.fq.gz 8658_1.fq.gz 8658_2.fq.gz 8672_1.fq.gz 8672_2.fq.gz"
LibHair_files = "34_1_wgs.fq.gz 34_2_wgs.fq.gz 7331_1_wgs.fq.gz 7331_2_wgs.fq.gz 7393_1_wgs.fq.gz 7393_2_wgs.fq.gz 8717_1_wgs.fq.gz 8717_2_wgs.fq.gz 8724_1_wgs.fq.gz 8724_2_wgs.fq.gz 8997_1_wgs.fq.gz 8997_2_wgs.fq.gz"

In [18]:
NlaIIIxMluCI_samples = [re.sub("_1.fq.gz", "", x) for x in NlaIIIxMluCI_files.split(" ") 
                        if re.search("_1.fq.gz", x)]
LibHair_samples = [re.sub("_1_wgs.fq.gz", "", x) for x in LibHair_files.split(" ") 
                                       if re.search("_1_wgs.fq.gz", x)]

In [19]:
NlaIIIxMluCI_samples[:5]

['10', '204', '312', '313', '377']

In [20]:
LibHair_samples

['34', '7331', '7393', '8717', '8724', '8997']

In [21]:
d = {}
d["library_name"] = "2014_02_XX_Burramys_MtBuller_RADSeq_NlaIIIxMluCI"
d["samples"] = NlaIIIxMluCI_samples
library_info["2014_02_XX_Burramys_MtBuller_RADSeq_NlaIIIxMluCI.zip"] = d

In [22]:
d = {}
d["library_name"] = "2017-12-26_Burramys_MtBuller_WGS_LibHair"
d["samples"] = LibHair_samples
library_info["2017-12-26_Burramys_MtBuller_WGS_LibHair.zip"] = d

-----

## Connect to MF server

In [23]:
with open("keys/key") as f:
    key = f.read().strip()
with open("keys/iv") as f:
    iv = f.read().strip()
obj = AES.new(key, AES.MODE_CFB, iv)

In [24]:
with open("/Users/jess/.ssh/encrypted_pw.txt") as f:
    pw = f.read().strip()

In [25]:
MF_HOST = "mediaflux.vicnode.org.au"
MF_PORT = 443
MF_TRANSPORT = "https"
MF_DOMAIN = "aaf"
MF_USER = "unimelb:jessicac"
MF_PASSWORD = obj.decrypt(pw)

In [26]:
con = mfclient.MFConnection(host=MF_HOST,
                            port=MF_PORT,
                            transport=MF_TRANSPORT,
                            domain=MF_DOMAIN,
                            user=MF_USER,
                            password=MF_PASSWORD)

In [27]:
logging.info("Connecting to mediaflux.")
con.open()
result = con.execute("server.version")

In [28]:
result.tostring()

'<result><ant-version>Apache Ant 1.9.4</ant-version><binary>aserver</binary><build-time>31-Jan-2018 16:49:25 AEDT</build-time><built-by>Arcitecta. Pty. Ltd.</built-by><created-by>1.8.0_111-b14 (Oracle Corporation)</created-by><manifest-version>1.0</manifest-version><target-jvm>1.7</target-jvm><vendor>Arcitecta Pty. Ltd.</vendor><version>4.6.034</version></result>'

-----

## Find sequencing archives on Mediaflux

Find sequencing archives and get asset ID on the Mediaflux server

In [29]:
# Query for RADSeq directory
args = mfclient.XmlStringWriter("args")
args.add("where", "namespace=/projects/proj-marsupial_genomics-1128.4.19/Burramys/RADSeq")
args.add("action", "get-path")
args.add("size", "infinity")
radseq_query = con.execute("asset.query", args.doc_text())

In [30]:
# Query for WGS directory
args = mfclient.XmlStringWriter("args")
args.add("where", "namespace=/projects/proj-marsupial_genomics-1128.4.19/Burramys/WGS")
args.add("action", "get-path")
args.add("size", "infinity")
wgs_query = con.execute("asset.query", args.doc_text())

In [31]:
wgs_query.tostring()

'<result><path id="35533853" version="1">/projects/proj-marsupial_genomics-1128.4.19/Burramys/WGS/2017-12-26_Burramys_MtBuller_WGS_LibHair.zip</path></result>'

In [32]:
# Add to asset ID to library_info
path_elements = radseq_query.elements("path") + wgs_query.elements("path")

for pe in path_elements:
    path = pe.value()
    id = pe.value("@id")
    archive_name = os.path.basename(path)
    print(id + " - " + archive_name)
    assert(archive_name in library_info)
    library_info[archive_name]["asset_id"] = id
    library_info[archive_name]["asset_path"] = path

35531098 - 2017-08-29_Burramys_MtBuller_RADSeq_Lib1.zip
35531152 - 2017-03-24_Burramys_MtBuller_RADSeq_Lib1_trial.zip
35532447 - 2014_02_XX_Burramys_MtBuller_RADSeq_NlaIIIxMluCI.zip
35532450 - 2016-10-30_Burramys_MtBuller_RADSeq_MiSeq.zip
35532451 - 2017-08-29_Burramys_MtBuller_RADSeq_Lib2.zip
35532452 - 2017-08-29_Burramys_MtBuller_RADSeq_Lib3.zip
35532453 - 2017-08-29_Burramys_MtBuller_RADSeq_Lib4.zip
35532454 - 2017-08-29_Burramys_MtBuller_RADSeq_Lib5.zip
35532456 - 2017-08-29_Burramys_MtBuller_RADSeq_Lib6.zip
35532457 - 2017-08-29_Burramys_MtBuller_RADSeq_Lib7.zip
35532458 - 2017-10-31_Burramys_MtBuller_RADSeq_Lib8.zip
35532460 - 2017-10-31_Burramys_MtBuller_RADSeq_Lib9.zip
35532461 - 2017-10-31_Burramys_MtBuller_RADSeq_Lib10.zip
35532462 - 2018-01-31_Burramys_MtBuller_RADSeq_Lib11.zip
35532463 - 2018-01-31_Burramys_MtBuller_RADSeq_Lib12.zip
35533853 - 2017-12-26_Burramys_MtBuller_WGS_LibHair.zip


-----

## Process barcode and sample names

Wrangle barcodes and sample names into a format that will be attached to the sequencing archive

In [33]:
for archive_name, info in library_info.iteritems():
    print(archive_name)
    metadata = []
    if "barcode_info" in info:
        for i in info["barcode_info"].itertuples():
            metadata.append("{},{},{}".format(i[1], i[2], i[3]))
    else:
        for i in info["samples"]:
            metadata.append(",,{}".format(i))
    library_info[archive_name]["metadata"] = metadata

2017-10-31_Burramys_MtBuller_RADSeq_Lib9.zip
2017-10-31_Burramys_MtBuller_RADSeq_Lib8.zip
2016-10-30_Burramys_MtBuller_RADSeq_MiSeq.zip
2017-10-31_Burramys_MtBuller_RADSeq_Lib10.zip
2017-08-29_Burramys_MtBuller_RADSeq_Lib7.zip
2017-08-29_Burramys_MtBuller_RADSeq_Lib6.zip
2017-12-26_Burramys_MtBuller_WGS_LibHair.zip
2017-08-29_Burramys_MtBuller_RADSeq_Lib2.zip
2017-08-29_Burramys_MtBuller_RADSeq_Lib4.zip
2018-01-31_Burramys_MtBuller_RADSeq_Lib12.zip
2017-08-29_Burramys_MtBuller_RADSeq_Lib1.zip
2014_02_XX_Burramys_MtBuller_RADSeq_NlaIIIxMluCI.zip
2017-03-24_Burramys_MtBuller_RADSeq_Lib1_trial.zip
2018-01-31_Burramys_MtBuller_RADSeq_Lib11.zip
2017-08-29_Burramys_MtBuller_RADSeq_Lib3.zip
2017-08-29_Burramys_MtBuller_RADSeq_Lib5.zip


In [34]:
library_info["2017-12-26_Burramys_MtBuller_WGS_LibHair.zip"]["metadata"]

[',,34', ',,7331', ',,7393', ',,8717', ',,8724', ',,8997']

In [35]:
library_info["2018-01-31_Burramys_MtBuller_RADSeq_Lib11.zip"]["metadata"][:10]

['ACGTCA,CATGAC,928',
 'GTACTG,CATGAC,934',
 'CTAGTC,CATGAC,975',
 'AGCTGA,CATGAC,1413',
 'TCGACT,CATGAC,1447',
 'GATCAG,CATGAC,1451',
 'ATGCTA,CATGAC,1461',
 'GCATAG,CATGAC,1470',
 'CGTAGC,CATGAC,1472',
 'TACGCT,CATGAC,1473']

-----

## Create metadata document

In [170]:
# test metadata has example
args = mfclient.XmlStringWriter("args")
args.add("type", "proj-marsupial_genomics-1128.4.19:test")
result = con.execute("asset.doc.type.describe", args.doc_text())

In [172]:
print result

<result><type for="asset" id="241" name="proj-marsupial_genomics-1128.4.19:test" version="8"><label>proj-marsupial_genomics-1128.4.19:test</label><description>test description</description><generated-by>user</generated-by><allow-incomplete>maybe</allow-incomplete><access><administer>true</administer><create>true</create><modify>true</modify><publish>true</publish></access><creator id="1398"><domain>aaf</domain><user>jchung@unimelb.edu.au</user><name>Jessica Chung</name><email>jchung@unimelb.edu.au</email></creator><ctime dst="true" gmt-offset="10.0" time="1517807989341" tz="Australia/Melbourne">05-Feb-2018 16:19:49</ctime><definition><element label="ID" max-occurs="1" name="id" type="string"><description>description for id</description><instructions>instructions for id</instructions></element><element max-occurs="100" min-occurs="0" name="location" type="string" /><element max-occurs="1" name="enum" type="enumeration"><restriction base="enumeration"><value>val1</value><value>val2</valu

In [ ]:
# <element max-occurs="1000" min-occurs="0" name="sample_barcodes" type="string">
#     <description>barcode and sample name in archive</description>
#     <instructions>Barcodes and sample name in the format barcode1:barcode2:sample_name (e.g. ACGTCA:CATGAC:8162)</instructions>
# </element>

In [187]:
project_namespace = "proj-marsupial_genomics-1128.4.19"
document_name = "mpp_seq_archive"

args = mfclient.XmlStringWriter("args")
args.add("type", "{}:{}".format(project_namespace, document_name))
args.add("label", "{}:{}".format(project_namespace, document_name))
args.add("description", "Sequence library information")
args.add("instructions", 
         "This metadata document holds information of a library and " \
         "should be attached to a zipped archive that contains sequencing " \
         "files in fastq format.")

# Metadata definitions
args.push("definition")

# Library name
attr = {"name": "library_name",
        "max-occurs": 1,
        "min-occurs": 1,
        "type": "string"}
args.push("element", attributes=attr)
args.add("description", "Library name of the data")
args.add("instructions", "e.g. '2017-08-29_Burramys_MtBuller_RADSeq_Lib1'")
args.pop()

# Barcodes and sample name
attr = {"name": "sample_barcode",
        "max-occurs": 1000,
        "min-occurs": 1,
        "type": "string"}
args.push("element", attributes=attr)
args.add("description", "Barcode and sample names included in the archive")
args.add("instructions", "Comma-separated values: barcode1,barcode2,sample_name (e.g. 'ACGTCA,CATGAC,8162')")
args.pop()

args.pop() # end definition

In [188]:
args.doc_text()

'<args><type>proj-marsupial_genomics-1128.4.19:mpp_seq_archive</type><label>proj-marsupial_genomics-1128.4.19:mpp_seq_archive</label><description>Sequence library information</description><instructions>This metadata document holds information of a library and should be attached to a zipped archive that contains sequencing files in fastq format.</instructions><definition><element min-occurs="1" type="string" name="library_name" max-occurs="1"><description>Library name of the data</description><instructions>e.g. \'2017-08-29_Burramys_MtBuller_RADSeq_Lib1\'</instructions></element><element min-occurs="1" type="string" name="sample_barcode" max-occurs="1000"><description>Barcode and sample names included in the archive</description><instructions>Comma-separated values: barcode1,barcode2,sample_name (e.g. \'ACGTCA,CATGAC,8162\')</instructions></element></definition></args>'

In [189]:
logging.info("Creating metadata document: {}:{}".format(project_namespace, document_name))
result = con.execute("asset.doc.type.create", args.doc_text())
logging.info(result)

-----

## Populate metadata

Set metadata for each library archive

In [37]:
project_namespace = "proj-marsupial_genomics-1128.4.19"
document_name = "mpp_seq_archive"
action = "merge"

In [46]:
for library_name, info in library_info.iteritems():
    print info["asset_id"] + " - " + library_name
#     print info["metadata"]
    args = mfclient.XmlStringWriter("args")
    args.add("id", info["asset_id"])
    args.push("meta", attributes={"action": action})
    args.push("{}:{}".format(project_namespace, document_name))
    args.add("library_name", info["library_name"])
    for x in info["metadata"]:
        args.add("sample_barcode", x)
    args.pop() # end namespace:metadata_document
    args.pop() # end meta
    logging.info("Setting metadata for asset id {} ({})".format(info["asset_id"], info["library_name"]))
    result = con.execute("asset.set", args.doc_text())

35532460 - 2017-10-31_Burramys_MtBuller_RADSeq_Lib9.zip
35532458 - 2017-10-31_Burramys_MtBuller_RADSeq_Lib8.zip
35532450 - 2016-10-30_Burramys_MtBuller_RADSeq_MiSeq.zip
35532461 - 2017-10-31_Burramys_MtBuller_RADSeq_Lib10.zip
35532457 - 2017-08-29_Burramys_MtBuller_RADSeq_Lib7.zip
35532456 - 2017-08-29_Burramys_MtBuller_RADSeq_Lib6.zip
35533853 - 2017-12-26_Burramys_MtBuller_WGS_LibHair.zip
35532451 - 2017-08-29_Burramys_MtBuller_RADSeq_Lib2.zip
35532453 - 2017-08-29_Burramys_MtBuller_RADSeq_Lib4.zip
35532463 - 2018-01-31_Burramys_MtBuller_RADSeq_Lib12.zip
35531098 - 2017-08-29_Burramys_MtBuller_RADSeq_Lib1.zip
35532447 - 2014_02_XX_Burramys_MtBuller_RADSeq_NlaIIIxMluCI.zip
35531152 - 2017-03-24_Burramys_MtBuller_RADSeq_Lib1_trial.zip
35532462 - 2018-01-31_Burramys_MtBuller_RADSeq_Lib11.zip
35532452 - 2017-08-29_Burramys_MtBuller_RADSeq_Lib3.zip
35532454 - 2017-08-29_Burramys_MtBuller_RADSeq_Lib5.zip


-----

## Close connection to Mediaflux

In [47]:
logging.info("Closing connection to mediaflux.")
con.close()